In [1]:
import numpy as np

In [2]:
from forkED.generator import *

In [3]:
folder = '/Users/ryansaxe/Projects/ML/forkED/data/ml-25m/processed/'
import pickle
dim = max(pickle.load(
    open(folder + 'maps/movie.pkl','rb')
).values()) + 1

In [4]:
dim

45251

In [5]:
import pickle
ddict = pickle.load(open(folder + 'sparse_lookups/users.pkl','rb'))

In [6]:
for k,v in ddict.items():
    new_arr = np.ones_like(v[1])
    new_arr[np.where(v[1] < 4)] = -1
    ddict[k] = (v[0],new_arr)

In [30]:
test = Augmentation(
    ddict,
    512,
    dim,
    verbose=False,
    file_cap_for_debug=10000,
    noise = (0.0, 0.0)
)

In [31]:
from forkED.model import EncoderDecoder

In [32]:
model = EncoderDecoder(
    dim,
    256,
    32,
    3,
    dropout=0.3,
    out_act = tf.nn.tanh
)

In [33]:
import tensorflow as tf
model.loss = tf.losses.mean_squared_error
model.optimizer = tf.optimizers.Adam(lr=0.001)

In [34]:
from forkED.trainer import Trainer

In [35]:
t = Trainer(test, model)

In [36]:
t.train(20)

Epoch 15/20:  30%|███       | 6/20 [00:06<00:14,  1.04s/Batch, loss=0.00287]

KeyboardInterrupt: 

In [42]:
example = test.storage[test.files[-1]]
user = np.zeros((1,test.dim))
user[0,example[0]] = example[1]

In [43]:
recs =  model(tf.convert_to_tensor(user,tf.float32))

In [45]:
recs.numpy().max()

0.8648552

In [46]:
recs.numpy().min()

-0.9996219